In [90]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

In [91]:
df = pd.read_csv('GB_Cleaned_dataset.csv')
df.columns

Index(['customer_number', 'Division', 'Customer_Type1', 'Customer_Type',
       'YearMonth', 'Monthly_SKUs_Purchased', 'Monthly_Vendors_Purchased',
       'Monthly_Number_of_Sales', 'Monthly_Last_Sales_Date',
       'Monthly_First_Sales_Date', 'Monthly_Sales_Days', 'Monthly_Sum_Sales',
       'Monthly_Gross_Margin', 'new_yearMonth', 'difference', 'churn',
       'Backlog_Sales'],
      dtype='object')

In [92]:
X = df[['Division', 'Customer_Type', 'Monthly_SKUs_Purchased', 'Monthly_Vendors_Purchased', 'Monthly_Number_of_Sales']]
Y = df['churn']


In [93]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 4)

In [94]:
x_train= Normalizer().fit(x_train).transform(x_train)
x_train= pd.DataFrame(x_train, columns = X.columns)
x_test = Normalizer().fit(x_test).transform(x_test)
x_test = pd.DataFrame(x_test, columns = X.columns)

In [95]:
y_train = list(y_train)
x_train = sm.add_constant(x_train)
LRmodel = sm.Logit(y_train, x_train).fit()
print(LRmodel.summary())

/Users/jamestao/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Optimization terminated successfully.
         Current function value: 0.638567
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               272403
Model:                          Logit   Df Residuals:                   272397
Method:                           MLE   Df Model:                            5
Date:                Mon, 06 Dec 2021   Pseudo R-squ.:                 0.07136
Time:                        18:31:06   Log-Likelihood:            -1.7395e+05
converged:                       True   LL-Null:                   -1.8731e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.4211      0.044    -54.979      0.000      -2.

In [96]:
y_probs = LRmodel.predict(x_train)
y_pred = list(map(round, y_probs))
print('training accuracy is',accuracy_score(y_train, y_pred))
y_test = list(y_test)
x_test = sm.add_constant(x_test)
y_probs_test = LRmodel.predict(x_test)
y_pred_test = list(map(round, y_probs_test))
print('testing accuracy is',accuracy_score(y_test, y_pred_test))

training accuracy is 0.6347140082891892
testing accuracy is 0.6355389952460491


/Users/jamestao/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [97]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, y_pred_test, labels=[0,1]))

[[44395 20055]
 [22494 29801]]


In [98]:
df_out = pd.DataFrame()
df_out['churn'] = y_test

In [99]:
df_out['Prob']=y_probs_test
df_out=df_out[['churn','Prob']]

In [100]:
df_out.to_csv("LR_TestingOutput.csv", index=False)